In [ ]:
import pandas as pd
import requests
import json
import os

In [ ]:
main_task = pd.read_csv('main_task.csv')
kaggle_task = pd.read_csv('kaggle_task.csv')
df = kaggle_task.append(main_task, sort=False).reset_index(drop=True)

In [ ]:
# Parsing

from lxml import html
from bs4 import BeautifulSoup
from multiprocessing import Pool
from datetime import datetime

TRIP_ADVISOR_URL_TEMPLATE = 'https://www.tripadvisor.com{}'


def parse_ratings_and_reviews(node, result):
    rating_block = node.find('div').findAll('div', recursive=False)[2]
    rating_block = rating_block.findAll('div', recursive=False)
    if len(rating_block) < 2:
        return

    ratings = rating_block[1].findAll('div')
    for rating in ratings:
        spans = rating.findAll('span', recursive=False)
        title = spans[1].text.lower()
        value = spans[2].find('span').attrs['class'][1].split('_')[1]
        result[title] = int(value)


def parse_location_and_contact(node):
    location_block = node.find('div').find('div')
    location_block = location_block.findAll('div', recursive=False)[1]
    distance_el = location_block.find('b')
    if distance_el is None:
        return np.NaN
    return float(distance_el.text.split()[0])


def parse_details_block(node, result):
    if node is None:
        return

    result['is_verified'] = 1 if node.find(
        'span', {'class': 'ui_icon verified-checkmark'}) is not None else 0
    result['has_phone_number'] = 1 if node.find(
        'a', string='+ Add phone number') is None else 0
    result['has_hours'] = 1 if node.find('a',
                                         string='+ Add hours') is None else 0
    result['has_website'] = 1 if node.find(
        'a', string='+ Add website') is None else 0
    result['has_menu'] = 1 if node.find('a', string='Menu') is not None else 0


def collect_page_data(html, result):
    soup = BeautifulSoup(html)
    overview_tabs = soup.find('div', {'data-tab': 'TABS_OVERVIEW'})
    if overview_tabs is None:
        return

    overview_columns = overview_tabs.findAll('div', {'class': 'ui_column'})
    parse_ratings_and_reviews(overview_columns[0], result)
    parse_details_block(overview_columns[1], result)

    result['distance'] = parse_location_and_contact(overview_columns[2])
    result['has_tcAward'] = 1 if soup.find(
        'img', {'class': 'tcAward'}) is not None else 0


def current_time():
    return datetime.now().strftime("%H:%M:%S")


def get_id_from_url(url):
    return url.split('-')[2]


def parse_site_page(url):
    result = {}
    result['id_ta'] = get_id_from_url(url)
    ta_url = TRIP_ADVISOR_URL_TEMPLATE.format(url)
    print(url)
    r = requests.get(ta_url, stream=True)
    print('Done')
    collect_page_data(r.text, result)
    return result


def process_ta_urls(x):
    return x.apply(parse_site_page)

In [ ]:
def parallelize_processing(df, func, n_cores=8):
    pool = Pool(n_cores)
    df_split = np.array_split(df, n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df


print("Started at:", current_time())

start = 0
step = 5000
stop = 50040
n_cores = 16

for i in range(start, stop, step):
    print('[{}] Getting records {}:{}'.format(current_time(), i, i + step))
    pages_data = parallelize_processing(data['URL_TA'][i:i + step],
                                        process_ta_urls,
                                        n_cores=n_cores)
    site_df = pd.DataFrame(pages_data.values.tolist())
    site_df.to_csv('ta_data/data_{}-{}.csv'.format(i, i + step))

print("Finished at:", current_time())

In [ ]:
# Выгрузим ссылки на страницы в отдельные файлы для работы граббера
start = 0
step = 5000
stop = len(df)

for i in range(start, stop, step):
    df[['ID_TA', 'URL_TA'
        ]][i:i + step].to_csv('./data/urls/urls_{}-{}.csv'.format(i, i + step),
                              header=False)

In [ ]:
main_task = pd.read_csv(
    '/Users/fancier/Downloads/skillfactory_rds-master/module_03/kaggle/grabber_data_50000/urls/urls_0-5000.csv'
)
main_task

In [ ]:
main_task.info()

In [ ]:
# Превратим набор JSON-файлов в CSV-формат для дальнейшего удобства рабты с ними
pieces = []

for dirname, _, filenames in os.walk(
        '/Users/fancier/Downloads/skillfactory_rds-master/module_03/kaggle/grabber_data_50000/ta_data'
):
    for filename in filenames:
        if not filename.endswith('.json'):
            continue

        with open(os.path.join(dirname, filename)) as f:
            data_pieces = json.load(f)
            pieces.append(pd.DataFrame(data_pieces))

data_tripadvisor = pd.concat(pieces, ignore_index=True)
data_tripadvisor.info()
data_tripadvisor.sample(10)

In [ ]:
for col in data_tripadvisor.columns:
    pct_missing = np.mean(data_tripadvisor[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing * 100)))

In [ ]:
data_tripadvisor.fillna(
    {
        'food': 0,
        'service': 0,
        'value': 0,
        'is_verified': 0,
        'has_phone_number': 0,
        'has_hours': 0,
        'has_website': 0,
        'has_menu': 0,
        'has_tcAward': 0
    },
    inplace=True)

data_tripadvisor.info()

In [ ]:
data_tripadvisor['has_tcAward'].value_counts()

In [ ]:
data_tripadvisor['distance_isNAN'] = pd.isna(
    data_tripadvisor['distance']).astype('uint8')
data_tripadvisor.fillna({'distance': data_tripadvisor['distance'].median()},
                        inplace=True)

In [ ]:
len(data_tripadvisor)

In [ ]:
for col in data_tripadvisor.columns:
    print(
        '\nColumn Name:',
        col,
    )
    print(data_tripadvisor[col].value_counts())

In [ ]:
data_tripadvisor.drop(
    columns=['id', 'ta_url', 'is_verified', 'has_menu', 'atmosphere'],
    inplace=True)
data_tripadvisor.info()

In [ ]:
data_tripadvisor.columns

In [ ]:
data_tripadvisor.columns = [
    'ID_TA', 'Food', 'Service', 'Value', 'Phone_Number', 'Hours', 'Website',
    'Distance', 'Award', 'Distance_isNAN'
]

In [ ]:
data_tripadvisor

In [ ]:
data_tripadvisor.to_csv('./data/data_tripadvisor_info.csv', index=False)

In [ ]:
# заведем словарь для гоордов с новыми данными [кол-во тыс. туристов, место в рейтенге благосостояния] по данным из wiki
city_info = {
    'London' : [19233, 14],
    'Paris' : [17560, 18],
    'Madrid' : [5440, 19],
    'Barcelona' : [6714, 19],
    'Berlin' : [5959, 15],
    'Milan' : [6481, 24],
    'Rome' : [10065, 24],
    'Prague' : [8949, 22],
    'Lisbon' : [3539, 29],
    'Vienna' : [6410, 2],
    'Amsterdam' : [8354, 7],
    'Brussels' : [3942, 13],
    'Hamburg' : [1450, 15],
    'Munich' : [4067, 15],
    'Lyon' : [6000, 18],
    'Stockholm' : [2605, 8],
    'Budapest' : [3823, 31],
    'Warsaw' : [2850, 27],
    'Dublin' : [5213, 16],
    'Copenhagen' : [3070, 5],
    'Athens' : [5728, 36],
    'Edinburgh' : [1660, 14],
    'Zurich' : [2240, 6],
    'Oporto' : [2341, 29],
    'Geneva' : [1150, 6],
    'Krakow' : [2732, 27],
    'Oslo' : [1400, 1],
    'Helsinki' : [1240, 9],
    'Bratislava' : [126, 26],
    'Luxembourg' : [1139, 11],
    'Ljubljana' : [5900, 20]
}

In [ ]:
city_info['Ljubljana'][0]

In [ ]:
tourists = data.City.apply(lambda x: th[x][0])
tourists.name = 'tourists'

In [ ]:
city

In [ ]:
city = pd.read_csv('cities.csv')
#city['tourists'] = city['City'].apply(lambda x: city_info.get(x))

In [ ]:
city_info = pd.DataFrame(city_info).T

In [ ]:
city_info.columns = [
    'Number_of_Tourists', 'Index_Happy', 'age', 'address', 'famsize',
    'parents_status', 'mum_edu', 'father_edu', 'mum_job', 'father_job',
    'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup',
    'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet',
    'romantic', 'famrel', 'freetime', 'goout', 'health', 'absences', 'score'
]

In [ ]:
import xlwings as xw
import pandas as pd

inn = xw.Book('0d9fffcb-ca81-5ee9-8a43-44151c7463dc.xls')
data_excel = inn.sheets['Данные']
data_pd = data_excel.range('A1:D7').options(pd.DataFrame,
                                            header=1,
                                            index=False).value
print(data_pd)

https://www.innovation-cities.com/index-2019-global-city-rankings/18842/

In [ ]:
inn = pd.read_csv('Innovation_Cities_Index.csv')